<a href="https://colab.research.google.com/github/sydneysimmons/Transformers-Project/blob/main/table_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs

In [1]:
!pip3 install transformers
# !pip install torch torchvision torchaudio
# !conda install pytorch-cpu torchvision-cpu -c pytorch
# !conda install -c pytorch pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.4 MB/s 
     |████████████████████████████████| 7.6 MB 35.0 MB/s 
     |████████████████████████████████| 182 kB 19.2 MB/s 


In [2]:
import torch

In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 106 kB 5.4 MB/s 
  Created wheel for torch-scatter: filename=torch_scatter-2.1.0-cp37-cp37m-linux_x86_64.whl size=305425 sha256=aa95d1d3ffacae5c00459d740fe477aca83c4676651b35f54955b812433d86e2
  Stored in directory: /root/.cache/pip/wheels/2d/d1/15/8a2f0086896d156654a843fff4bdbeaf621cdd10310a0daad2
Successfully built torch-scatter


# Imports

In [4]:
!pip install transformers
# !conda install -c conda-forge transformers -y
from transformers import pipeline
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
tqa = pipeline(task="table-question-answering", 
               model="google/tapas-base-finetuned-wtq")

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

# Reading Data

In [7]:
!git clone https://github.com/sydneysimmons/Transformers-Project.git

Cloning into 'Transformers-Project'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [8]:
pwd

'/content'

In [9]:
table = pd.read_csv("/content/Transformers-Project/data.csv")
table = table.astype(str)

In [10]:
table

,Rk,Player,Team,Age,Position,Games,Games Started,QBrec,Completions,Attempts,...,Y/G,Rate,Quarterback Rating,Sk,Yds,Sk%,NY/A,ANY/A,4QC,Game Winning Drives
0,1,Tom Brady*,TAM,44,QB,17,17,13-4-0,485,719,...,312.7,102.1,68.1,22,144,3.0,6.98,7.41,3.0,5.0
1,2,Justin Herbert*,LAC,23,QB,17,17,9/8/00,443,672,...,294.9,97.7,65.6,31,214,4.4,6.83,6.95,5.0,5.0
2,3,Matthew Stafford,LAR,33,QB,17,17,12/5/00,404,601,...,287.4,102.9,63.8,30,243,4.8,7.36,7.45,3.0,4.0
3,4,Patrick Mahomes*,KAN,26,QB,17,17,12/5/00,436,658,...,284.6,98.5,62.2,28,146,4.1,6.84,7.07,3.0,3.0
4,5,Derek Carr,LVR,30,QB,17,17,10/7/00,428,626,...,282.6,94.0,52.4,40,241,6.0,6.85,6.6,3.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,Cordarrelle Patterson,ATL,30,RB,16,13,nan,0,1,...,0.0,39.6,2.9,0,0,0.0,0.0,0.0,nan,nan
111,112,Brett Rypien,DEN,25,QB,1,0,nan,0,2,...,0.0,39.6,1.1,0,0,0.0,0.0,0.0,nan,nan
112,113,D'Andre Swift,DET,22,RB,13,4,nan,0,1,...,0.0,39.6,0.5,0,0,0.0,0.0,0.0,nan,nan
113,114,Albert Wilson,MIA,29,WR,14,5,nan,0,1,...,0.0,39.6,0.0,1,1,50.0,-0.5,-0.5,nan,nan


# Magic

In [11]:
query = "How many touchdowns did Aaron Rodgers throw?"
print(tqa(table=table, query=query)["answer"])

AVERAGE > 37


In [12]:
query = ["Who had the most TDs?",
         "Which team does he play for?"]
answer = tqa(table=table, query=query)
for ans in answer:
    print(ans["answer"])

Tom Brady*
TAM


# Training

In [ ]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace

# gets role for executing training job
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='{IAM_ROLE_WITH_SAGEMAKER_PERMISSIONS}')['Role']['Arn']
hyperparameters = {
    'model_name_or_path':'google/tapas-base-finetuned-wtq',
    'output_dir':'/opt/ml/model'
    # add your remaining hyperparameters
    
    # more info here https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/question-answering
}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.17.0'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    entry_point='run_qa.py',
    source_dir='./examples/pytorch/question-answering',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    git_config=git_config,
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    hyperparameters = hyperparameters
)

# starting the train job
huggingface_estimator.fit()